In [ ]:
import ROOT
from T2KStyle.T2KStyle import GetT2K
style = GetT2K(3)
ROOT.gROOT.SetStyle("T2K")
c = ROOT.TCanvas('c', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)

In [ ]:
path_loc = '~/DATA/'
path = '~/LXPLUS/DESY_testbeam/tree_v3/'
path_d = '~/LXPLUS/DESY_testbeam/tree_v7/'
path_c = '~/LXPLUS/DESY_testbeam/tree_v9/'
path_c2 = '~/LXPLUS/DESY_testbeam/tree_v12/'

# file_name = path + '/g_360_200_100p'
N_iter = 9
color = [ROOT.kRed, ROOT.kGreen, ROOT.kBlack, ROOT.kBlue, ROOT.kViolet, ROOT.kYellow, ROOT.kCyan, ROOT.kOrange, ROOT.kMagenta, ROOT.kAzure]

## Reslution vs angle

In [ ]:
%jsroot off
ROOT.gStyle.SetOptTitle(0)
c.cd()
# ROOT.gPad.SetGrid()

angles = [0, 10, 20, 30, 40, 45, 50, 60, 70, 80, 90]
angles_new = [20, 30, 40, 45, 50, 60, 70]
angle_new2 = [20, 30, 60, 70]
angle_new3 = [60]
# angles = [0, 10, 80, 90]
# angles_new = [40, 45, 50]
# angle_new2 = [20, 30, 60, 70]
# angle_new3 = [60]

peaking = 412

fit_low = -0.003
fit_high = 0.003

if peaking == 200:
    angles.remove(45)
    angles.remove(90)
    angles_new.remove(45)

gr = ROOT.TGraphErrors()
N_iter = 8
for angle in angles:
    file = ROOT.TFile(path + f'phi_{str(peaking)}_{str(angle)}_iter{N_iter}.root')
    resol = file.Get('resol_total')
    resol.Fit('gaus', 'Q', '', fit_low, fit_high)
    res = resol.GetFunction('gaus').GetParameter(2)
    res_e = resol.GetFunction('gaus').GetParError(2)
    gr.SetPoint(gr.GetN(), angle, res*1e6)
    gr.SetPointError(gr.GetN()-1, 0., res_e*1e6)
    
gr2 = ROOT.TGraphErrors()
print("diag")
for angle in angles_new:
    if angle in [40, 45, 50]:
        file = ROOT.TFile(path_d + f'phi_{str(peaking)}_{str(angle)}_d_iter{N_iter}.root')
    else:
        file = ROOT.TFile(path_d + f'phi_{str(peaking)}_{str(angle)}_d_iter{N_iter}.root')
    resol = file.Get('resol_total')
    resol.Fit('gaus', 'Q', '', fit_low, fit_high)
    res = resol.GetFunction('gaus').GetParameter(2)
    res_e = resol.GetFunction('gaus').GetParError(2)
    print(angle, res*1e6, file.outtree.GetEntries(), sep='\t')
    gr2.SetPoint(gr2.GetN(), angle, res*1e6)
    gr2.SetPointError(gr2.GetN()-1, 0., res_e*1e6)
    
gr3 = ROOT.TGraphErrors()
print('complicated')
for angle in angle_new2:
    file = ROOT.TFile(path_c + f'phi_{str(peaking)}_{str(angle)}_c_iter{N_iter}.root')
    resol = file.Get('resol_total')
    resol.Fit('gaus', 'Q', '', fit_low, fit_high)
    res = resol.GetFunction('gaus').GetParameter(2)
    res_e = resol.GetFunction('gaus').GetParError(2)
    print(angle, res*1e6, file.outtree.GetEntries(), sep='\t')
    gr3.SetPoint(gr3.GetN(), angle, res*1e6)
    gr3.SetPointError(gr3.GetN()-1, 0., res_e*1e6)
    
gr4 = ROOT.TGraphErrors()
print('complicated')
for angle in angle_new2:
    file = ROOT.TFile(path_c2 + f'/phi_{str(peaking)}_{str(angle)}_c_iter{N_iter}.root')
    resol = file.Get('resol_total')
    resol.Fit('gaus', 'Q', '', fit_low, fit_high)
    res = resol.GetFunction('gaus').GetParameter(2)
    res_e = resol.GetFunction('gaus').GetParError(2)
    print(angle, res*1e6, file.outtree.GetEntries(), sep='\t')
    gr4.SetPoint(gr4.GetN(), angle, res*1e6)
    gr4.SetPointError(gr4.GetN()-1, 0., res_e*1e6)
    
gr.Draw('ap')
gr.GetXaxis().SetTitle("#phi [deg]")
gr.GetYaxis().SetTitle("Resolution [#mum]")
gr.GetXaxis().SetRangeUser(-10., 100.)
gr.SetTitle('Row/column fit')
gr.SetMaximum(1500)
gr.SetMinimum(0.)

gr2.SetMarkerColor(ROOT.kRed)
gr2.Draw('same p')
gr2.SetMarkerStyle(21)
gr2.SetTitle('Diagonal fit')

gr3.SetMarkerColor(ROOT.kBlue)
gr3.SetTitle('2by1 & 3by1')
gr3.SetMarkerStyle(22)
gr3.Draw('same p')

gr4.SetMarkerColor(ROOT.kGreen)
gr4.SetTitle('2by1 & 3by1 separate PRF')
gr4.SetMarkerStyle(23)
# gr4.Draw('same p')

c.Draw()
ROOT.gStyle.SetLegendBorderSize(1);
leg = c.BuildLegend()
leg.SetName('Legend')
c.Print('SR_slope.pdf')

## Resolution vs iteration

### diagonals

In [ ]:
%jsroot on
c.Clear()
res_evo = []
mg = ROOT.TMultiGraph()
N_iter = 7
peaking = 200
angles = [30, 40, 50, 60, 70]
for f, angle in enumerate(angles):
    res_evo.append(ROOT.TGraphErrors())
    for it in range(N_iter):
        file = ROOT.TFile(path_d + f'phi_{str(peaking)}_{str(angle)}_d_iter{it}.root')
        res = file.Get('resol_total')
        res.Fit('gaus', 'Q')
        fit_res = res.GetFunction("gaus");
        sigma = fit_res.GetParameter(2);
        sigma_e = fit_res.GetParError(2);
        res_evo[-1].SetPoint(it, it, sigma*1e6)
        res_evo[-1].SetPointError(it-1, sigma_e) 
    res_evo[-1].SetTitle(f'{angle}' + '^{#circ}')
    res_evo[-1].SetMarkerColor(color[f])
    mg.Add(res_evo[-1])
    

mg.GetXaxis().SetRangeUser(-1, 10)
mg.GetXaxis().SetTitle('Iteration')
mg.GetYaxis().SetTitle('Resolution [#mum]')
mg.SetMinimum(0.)
mg.SetMaximum(1700.)
mg.Draw('apl')
c.BuildLegend()
c.SetGrid()
c.Draw()

### Row/column

In [ ]:
%jsroot on
c.Clear()
res_evo = []
mg = ROOT.TMultiGraph()
N_iter = 4
peaking = 412
angles = [0, 10, 20, 30, 40, 45, 50, 60, 70, 80]
for f, angle in enumerate(angles):
    res_evo.append(ROOT.TGraphErrors())
    for it in range(N_iter):
        if angle == 0:
            file = ROOT.TFile(path + f'phi_{str(peaking)}_00_iter{N_iter}.root')
        else:
            file = ROOT.TFile(path + f'phi_{str(peaking)}_{str(angle)}_iter{it}.root')
        res = file.Get('resol_total')
        res.Fit('gaus', 'Q')
        fit_res = res.GetFunction("gaus");
        sigma = fit_res.GetParameter(2);
        sigma_e = fit_res.GetParError(2);
        res_evo[-1].SetPoint(it, it, sigma*1e6)
        res_evo[-1].SetPointError(it-1, sigma_e) 
    res_evo[-1].SetTitle(f'{angle}' + '^{#circ}')
    res_evo[-1].SetMarkerColor(color[f])
    mg.Add(res_evo[-1])
    

mg.GetXaxis().SetRangeUser(-1, 10)
mg.GetXaxis().SetTitle('Iteration')
mg.GetYaxis().SetTitle('Resolution [#mum]')
mg.SetMinimum(0.)
mg.SetMaximum(1700.)
mg.Draw('apl')
c.BuildLegend()
c.SetGrid()
c.Draw()

## Track mean RMS

In [ ]:
it = 7
file = ROOT.TFile(path_d + '/phi_412_45_d_iter' + str(it) + '.root')
# file = ROOT.TFile(path_loc + '/g_360_200_iter' + str(it) + '.root')
ROOT.gStyle.SetOptStat(0)
cl.cd()
cl.Clear()
cl.Divide(2, 2)
h = ROOT.TH1F('h', '', 200, 0., 2.)
hm = ROOT.TH1F('hm', '', 200, 0., 1e-4)
h2 = ROOT.TH2F('h2', '', 200, 0., 2., 100, 0., 10000.)
hc = ROOT.TH1F('hc', '', 50, 0., 70.)
for entry in file.outtree:
    mean = 0
    n = 0
    max_q = 0
    dedx = 0
    for i in range(len(entry.charge)):
        res = entry.residual[i]
        if res == -999:
            continue
        mean += entry.residual[i]*1e3
        n += 1
        dedx += entry.charge[i]
        if entry.charge[i] > max_q:
            max_q = entry.charge[i]
    if n == 0:
        continue
    mean /= n
    rms = 0
    for i in range(len(entry.charge)):
        res = entry.residual[i]
        if res == -999:
            continue
        rms += (res*1e3 - mean)**2
    rms = rms**0.5
    rms /= n**0.5
    dedx /= n

    if (rms != 0):
        h.Fill(rms)
        hm.Fill(mean)
        h2.Fill(rms, max_q)
        hc.Fill(n)
        
cl.cd(1)
h.GetXaxis().SetTitle('Track residual RMS [mm]')
h.Draw('')
cl.cd(2)
h2.GetXaxis().SetTitle('Track residual RMS [mm]')
h2.GetYaxis().SetTitle('Max charge in track [a.u.]')
h2.Draw('colz')
cl.cd(3)
hm.GetXaxis().SetTitle('Track residual mean [mm]')
hm.Draw('')
cl.cd(4)
hc.GetXaxis().SetTitle("clusters used")
hc.Draw()

cl.Draw()

In [ ]:
%jsroot on
it = 4
c.cd()
h = []
hs = ROOT.THStack()
fo = ROOT.TFile("test.root", "RECREATE")

fl = []
title = []
angles = [0, 10, 80, 90]
angles_new = [40, 45, 50]
angle_new2 = [20, 30, 60, 70]

peaking = 412

fit_low = -0.003
fit_high = 0.003

if peaking == 200:
    angles.remove(45)
    angles.remove(90)
    angles_new.remove(45)

gr = ROOT.TGraphErrors()
N_iter = 8
for angle in angles:
    fl.append(ROOT.TFile(path + f'phi_{str(peaking)}_{str(angle)}_iter{N_iter}.root'))
    title.append(str(angle))
for angle in angles_new:
    fl.append(ROOT.TFile(path_d + f'phi_{str(peaking)}_{str(angle)}_d_iter{N_iter}.root'))
    title.append(str(angle))
for angle in angle_new2:
    fl.append(ROOT.TFile(path_c2 + f'/phi_{str(peaking)}_{str(angle)}_c_iter{N_iter}.root'))
    title.append(str(angle))

for f in range(30):
    h.append(ROOT.TH1F(f'h_{f}', '', 100, 0., 2.))

for f in range(len(fl)):
    print('Working on file', title[f])
#     file = ROOT.TFile(fl[f] + '_iter' + str(it) + '.root')
    for entry in fl[f].outtree:
        file.cd()
        mean = 0
        n = 0
        max_q = 0
        dedx = 0
        for i in range(len(entry.charge)):
            res = entry.residual[i]
            if res == -999:
                continue
            mean += entry.residual[i]*1e3
            n += 1
            dedx += entry.charge[i]
            if entry.charge[i] > max_q:
                max_q = entry.charge[i]
        if n == 0:
            print('zero n')
            continue
        mean /= n
        rms = 0
        for i in range(len(entry.charge)):
            res = entry.residual[i]
            if res == -999:
                continue
            rms += (res*1e3 - mean)**2
        rms = rms**0.5
        rms /= n**0.5
        dedx /= n
    
        if (rms != 0):
            fo.cd()
            h[f].Fill(rms)
    h[f].SetTitle(title[f])
    h[f].SetLineColor(ROOT.kBlack+f)
    print(title[f], h[f].GetMean())
    h[f].Scale(1./h[f].Integral())
    hs.Add(h[f].Clone(f'cl_{f}'))
        
c.cd()
hs.Draw('nostack histo')
hs.GetXaxis().SetTitle('Track residual RMS [mm]')
c.Draw()
c.BuildLegend()

## Looking at anomalies

In [ ]:
cw.cd()
cw.Clear()
cw.Divide(2)
rms_threshold = 20.
it = 2
file = ROOT.TFile(path + '/phi_412_40_iter' + str(it) + '.root')
h = ROOT.TH1F('h', '', 36, 0., 36.)
hc = ROOT.TH1F('hc', '', 36, 0., 36.)
found = 0
for entry in file.outtree:
    mean = 0
    n = 0
    max_q = 0
    dedx = 0
    for i in range(len(entry.charge)):
        res = entry.residual[i]
        if res == -999:
            continue
        mean += entry.residual[i]*1e3
        n += 1
        dedx += entry.charge[i]
        if entry.charge[i] > max_q:
            max_q = entry.charge[i]
    if n == 0:
        print('zero n')
        continue
    mean /= n
    rms = 0
    for i in range(len(entry.charge)):
        res = entry.residual[i]
        if res == -999:
            continue
        rms += (res*1e3 - mean)**2
    rms /= n**0.5
    dedx /= n
    if rms > rms_threshold:
        h.Reset()
        hc.Reset()
        for i in range(len(entry.charge)):
            if ( entry.residual[i] != -999):
                h.SetBinContent(i+1, entry.residual[i]*1e3)
                hc.SetBinContent(i+1, entry.charge[i])
        found += 1
    if found > 19:
        print(entry.ev)
        print(rms)
        break

cw.cd(1)
h.GetYaxis().SetTitle('Residual [mm]')
h.Draw('hist')
cw.cd(2)
hc.GetYaxis().SetTitle('Charge [a.u.]')
hc.Draw('hist')
cw.Draw()